In [1]:
cd /private/home/dpf/projects/codex

/private/home/dpf/projects/codex


In [2]:
import os
import pickle

In [3]:
from collections import Counter

In [4]:
from type_hints import normalize_type
from typewriter import evaluate

In [5]:
def load_results(dir_containing_shards):
    results = []
    for shard in range(10):
        with open(os.path.join(dir_containing_shards, f"shard_{shard}", f"results_shard-{shard}-of-10.pkl"), 'rb') as f:
            shard_results = pickle.load(f)
        results += shard_results
    print(f"loaded {len(results)} results")
    return results

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
instances = load_results("expts/typewriter/cm-6B_formatted_iandf_indent_def_ncg-1_temp-0.0_prompt-py/")

loaded 4168 results


In [8]:
inst = instances[0]

In [9]:
inst.keys()

dict_keys(['extra_left', 'left', 'right', 'return_type_from_source', 'true_type', 'typewriter_predicted_types', 'predicted_type', 'complete', 'prediction_untruncated'])

In [10]:
inst['true_type']

'List[Result]'

In [11]:
inst['predicted_type']

'List[Result]'

In [12]:
inst['typewriter_predicted_types']

['List[Result]', 'str', 'List[str]', '@@UNK@@', 'None']

In [13]:
print(normalize_type('@@UNK@@'))

could not parse type @@UNK@@
invalid syntax (<unknown>, line 1)
None


In [14]:
confusion_matrix = Counter()
verbose = True
t_correct_o_wrong_instances = []
for ix, inst in enumerate(instances):
    t_pred = normalize_type(inst['typewriter_predicted_types'][0])
    true = normalize_type(inst['return_type_from_source'])
    o_pred = normalize_type(inst['predicted_type'])
    t_correct = t_pred == true
    o_correct = o_pred == true
    confusion_matrix[f"t: {t_correct}, o: {o_correct}"] += 1
    if t_correct and not o_correct:
        t_correct_o_wrong_instances.append(inst)
    if verbose and ix % 80 == 0:
        print(f"{ix}\ttrue: {true}")
        print(f"{ix}\tours: {o_pred}\t{o_correct}")
        print(f"{ix}\ttywr: {t_pred}\t{t_correct}")
        print(inst['left'] + inst['right'])
        print()

0	true: List[Result]
0	ours: List[Result]	True
0	tywr: List[Result]	True


def get_header_issues(res, raw, url) -> :
    results = []

    try:
        headers = res.headers

        if "X-Powered-By" in headers:
            results.append(
                Result.from_evidence(
                    Evidence.from_response(res),
                    f'X-Powered-By Header Present: {headers["X-Powered-By"]} ({url})',
                    Vln.HTTP_HEADER_X_POWERED_BY,
                )
            )

            # check to see if this is a php version
            results += php.check_version(headers["X-Powered-By"], raw, url)

        if "X-XSS-Protection" in headers:
            # header is present, check the value
            if "0" in headers["X-XSS-Protection"]:
                results.append(
                    Result.from_evidence(
                        Evidence.from_response(res),
                        f"X-XSS-Protection Disabled Header Present ({url})",
                        Vln

In [60]:
def get_returns(function_body):
    lines = function_body.splitlines()
    split_lines = [line.strip().split() for line in lines]
    returns = [toks for toks in split_lines if toks and toks[0] == 'return']
    return returns

In [91]:
def get_non_none_returns(function_body):
    returns = get_returns(function_body)
    return [ret for ret in returns if len(ret) > 1 and ret[1:] != ['None']]

In [92]:
get_returns(instances[6]['right'])

[['return', 'req,', 'res'],
 ['return', 'body'],
 ['return', 'domain'],
 ['return', 'None'],
 ['return', 'None'],
 ['return', 'results']]

In [93]:
get_non_none_returns(instances[6]['right'])

[['return', 'req,', 'res'],
 ['return', 'body'],
 ['return', 'domain'],
 ['return', 'results']]

In [94]:
instances[0].keys()

dict_keys(['extra_left', 'left', 'right', 'return_type_from_source', 'true_type', 'typewriter_predicted_types', 'predicted_type', 'complete', 'prediction_untruncated'])

In [95]:
get_returns(instances[0]['right'])

[['return', 'results']]

In [102]:
def force_none(instance):
    non_none_returns = get_non_none_returns(instance['right'])
    if len(non_none_returns) == 0:
        instance = instance.copy()
        instance['predicted_type'] = 'None'
    return instance

In [97]:
get_non_none_returns(instances[0]['right'])

[['return', 'results']]

In [98]:
evaluate([force_none(instance) for instance in instances], type_from_source=True)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
Iterator[str]
None
None
Any
None
None
None
None
None
None
None
Iterator[None]
Iterator[None]
None
None
str
bool
List[str]
None
None
None
None
None
List[BaseRelation]
None
None
str
str
str
str
str
str
str
Any
None
None
str
Iterator[None]
None
Iterator[None]
None
Iterator[None]
NoReturn
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None

could not parse type Union[None, str, Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], 
unexpected EOF while parsing (<unknown>, line 1)
could not parse type message_update(self, message, new_subject, new_content, new_rendered_content, sender,
                       sender_realm, 
unexpected EOF while parsing (<unknown>, line 2)


{'precision': 0.7669227076332213,
 'recall': 0.7665547024952015,
 'f1': 0.7667386609071274,
 'n_correct': 3195,
 'n_predictions': 4166,
 'n_types': 4168}

In [99]:
evaluate(instances, type_from_source=True)

could not parse type Union[None, str, Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], 
unexpected EOF while parsing (<unknown>, line 1)
could not parse type message_update(self, message, new_subject, new_content, new_rendered_content, sender,
                       sender_realm, 
unexpected EOF while parsing (<unknown>, line 2)


{'precision': 0.46831493038886224,
 'recall': 0.4680902111324376,
 'f1': 0.4682025437964963,
 'n_correct': 1951,
 'n_predictions': 4166,
 'n_types': 4168}

In [104]:
evaluate([force_none(inst) for inst in instances], type_from_source=True)

could not parse type Union[None, str, Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], 
unexpected EOF while parsing (<unknown>, line 1)
could not parse type message_update(self, message, new_subject, new_content, new_rendered_content, sender,
                       sender_realm, 
unexpected EOF while parsing (<unknown>, line 2)


{'precision': 0.7669227076332213,
 'recall': 0.7665547024952015,
 'f1': 0.7667386609071274,
 'n_correct': 3195,
 'n_predictions': 4166,
 'n_types': 4168}

In [100]:
evaluate([inst for inst in instances if inst["return_type_from_source"] != 'None'], type_from_source=True, verbose=False)

could not parse type Union[None, str, Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], 
unexpected EOF while parsing (<unknown>, line 1)


{'precision': 0.5920612147297943,
 'recall': 0.5917782026768642,
 'f1': 0.591919674874492,
 'n_correct': 1238,
 'n_predictions': 2091,
 'n_types': 2092}

In [103]:
evaluate([force_none(inst) for inst in instances if inst["return_type_from_source"] != 'None'], type_from_source=True, verbose=False)

could not parse type Union[None, str, Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], 
unexpected EOF while parsing (<unknown>, line 1)


{'precision': 0.5523672883787661,
 'recall': 0.5521032504780115,
 'f1': 0.5522352378675592,
 'n_correct': 1155,
 'n_predictions': 2091,
 'n_types': 2092}

In [32]:
from typewriter import get_typewriter_predictions

In [46]:
# TypeWriter, all

In [47]:
evaluate(get_typewriter_predictions(instances), type_from_source=True, verbose=False)

{'precision': 0.7882639264467279,
 'recall': 0.6993761996161229,
 'f1': 0.741164505466565,
 'n_correct': 2915,
 'n_predictions': 3698,
 'n_types': 4168}

In [45]:
# Ours, all

In [44]:
evaluate(instances, type_from_source=True, verbose=False)

could not parse type Union[None, str, Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], 
unexpected EOF while parsing (<unknown>, line 1)
could not parse type message_update(self, message, new_subject, new_content, new_rendered_content, sender,
                       sender_realm, 
unexpected EOF while parsing (<unknown>, line 2)


{'precision': 0.46831493038886224,
 'recall': 0.4680902111324376,
 'f1': 0.4682025437964963,
 'n_correct': 1951,
 'n_predictions': 4166,
 'n_types': 4168}

In [39]:
# TypeWriter, non-None

In [33]:
evaluate(get_typewriter_predictions([inst for inst in instances if inst["return_type_from_source"] != 'None']), type_from_source=True, verbose=False)

{'precision': 0.5492700729927007,
 'recall': 0.43164435946462715,
 'f1': 0.48340471092077086,
 'n_correct': 903,
 'n_predictions': 1644,
 'n_types': 2092}

In [40]:
# Ours, non-None

In [37]:
evaluate([inst for inst in instances if inst["return_type_from_source"] != 'None'], type_from_source=True, verbose=False)

could not parse type Union[None, str, Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], Tuple[str], 
unexpected EOF while parsing (<unknown>, line 1)


{'precision': 0.5920612147297943,
 'recall': 0.5917782026768642,
 'f1': 0.591919674874492,
 'n_correct': 1238,
 'n_predictions': 2091,
 'n_types': 2092}

In [41]:
# TypeWriter, None

In [36]:
evaluate(get_typewriter_predictions([inst for inst in instances if inst["return_type_from_source"] == 'None']), type_from_source=True, verbose=False)

{'precision': 0.9795520934761441,
 'recall': 0.9691714836223507,
 'f1': 0.9743341404358353,
 'n_correct': 2012,
 'n_predictions': 2054,
 'n_types': 2076}

In [50]:
# Ours, None

In [51]:
evaluate([inst for inst in instances if inst["return_type_from_source"] == 'None'], type_from_source=True, verbose=False)

could not parse type message_update(self, message, new_subject, new_content, new_rendered_content, sender,
                       sender_realm, 
unexpected EOF while parsing (<unknown>, line 2)


{'precision': 0.3436144578313253,
 'recall': 0.34344894026974954,
 'f1': 0.34353167911346666,
 'n_correct': 713,
 'n_predictions': 2075,
 'n_types': 2076}